In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import normalize

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

from scipy.stats import ks_2samp

from matplotlib import pyplot as plt
%matplotlib inline

inputFolder='../data/'
trainFolder=inputFolder+'train/'
testFolder=inputFolder+'test/'

In [399]:
trainUsers=pd.read_csv(trainFolder+'user_info_train.txt', \
                           names=['id','sex','profession','education','marriage','citizen_type'],\
                          index_col=0)

trainBanks=pd.read_csv(trainFolder+'bank_detail_train.txt',\
                          names=['user_id', 'time', 'transaction_type', 'amount', 'is_salary'])

trainBrowse=pd.read_csv(trainFolder+'browse_history_train.txt',\
                           names=['user_id', 'time', 'browse_type', 'sub_type'])

trainBills=pd.read_csv(trainFolder+'bill_detail_train.txt',\
                          names=['user_id', 'time', 'bank_id', 'last_bill_amount', 'last_bill_pay', 'credit_line',\
                                 'cur_bill_balance', 'cur_bill_min_due', 'transactionNum', 'cur_bill_amount', \
                                 'adjusted_amount', 'cumulative_interest', 'available_deposit', 'available_credit', 'debt_status'])

trainLoanTime=pd.read_csv(trainFolder+'loan_time_train.txt',\
                                names=['user_id', 'time'])

trainOverdue=pd.read_csv(trainFolder+'overdue_train.txt',\
                            names=['user_id', 'label'], index_col=0)

In [9]:
print 

In [1151]:
userFeatures=trainUsers.copy()

bankRecordsPersonal=trainBanks.groupby('user_id')

userFeatures['transactionNum']=bankRecordsPersonal['amount'].count()
userFeatures['transactionAmount']=bankRecordsPersonal['amount'].sum()

# got worse
# userFeatures['transactionNetAmount']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==1]['amount'].sum())-\
#         bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==0]['amount'].sum())
# userFeatures['transactionTimeSpan']=bankRecordsPersonal['time'].max()-bankRecordsPersonal['time'].min()

userFeatures['transactionExpense']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==1]['amount'].sum())
userFeatures['transactionExpenseMax']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==1]['amount'].max())
userFeatures['transactionExpenseAvg']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==1]['amount'].mean())

userFeatures['transactionIncome']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==0]['amount'].sum())
# not useful
# userFeatures['transactionIncomeMax']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==0]['amount'].max())
# userFeatures['transactionIncomeAvg']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==0]['amount'].mean())

userFeatures['transactionExpenseNum']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==1]['amount'].count())
userFeatures['transactionIncomeNum']=bankRecordsPersonal.apply(lambda x: x[x['transaction_type']==0]['amount'].count())
userFeatures['salaryNum']=bankRecordsPersonal['is_salary'].sum()
userFeatures['salaryTotal']=bankRecordsPersonal.apply(lambda x: x[x['is_salary']==1]['amount'].sum())
userFeatures['salaryAvg']=bankRecordsPersonal.apply(lambda x: x[x['is_salary']==1]['amount'].mean())
userFeatures['salaryMax']=bankRecordsPersonal.apply(lambda x: x[x['is_salary']==1]['amount'].max())

# userFeatures['salaryAvg']=

browseRecordsPersonal=trainBrowse.groupby('user_id')
userFeatures['browseNum']=browseRecordsPersonal['time'].count()
userFeatures['browseTypes']=browseRecordsPersonal['browse_type'].nunique()
userFeatures['browseSubTypes']=browseRecordsPersonal.apply(lambda x: x.groupby('browse_type')['sub_type'].nunique().sum())


bills=trainBills.groupby('user_id')
userFeatures['billBanksNum']=bills['bank_id'].nunique()
userFeatures['billNum']=bills['time'].count()

userFeatures['avgCreditLine']=bills['credit_line'].mean()
userFeatures['avgTransactionNum']=bills['transactionNum'].mean()
userFeatures['maxBankCredit_line']=bills.apply(lambda x: x.groupby('bank_id')['credit_line'].mean().max())
# userFeatures['loanTime']=trainLoanTime['time']

In [1135]:
# userFeatures['try']=bills.apply(lambda x: 1.0*sum(x['last_bill_amount']-x['last_bill_pay'])/len(x))
# userFeatures['try']=bills.apply(lambda x: x.groupby('bank_id')['last_bill_amount'].mean().min())
# userFeatures['try']=bills.apply(lambda x: x.groupby('bank_id')['cumulative_interest'].mean().max())

print userFeatures[['try', 'billNum']].head(20)

             try  billNum
id                       
3150    0.000000     38.0
6965   15.753222      4.0
1265   10.749081     54.0
6360   16.326643    141.0
2583   14.786282     24.0
34764   0.000000     18.0
9554   18.655617     17.0
6720    0.000000      2.0
29165   0.000000      3.0
6966   14.371964    227.0
35421  12.567130     54.0
27162   0.000000      1.0
2443   15.923561    159.0
19579   0.000000    309.0
590     0.000000     11.0
42390  10.772794    104.0
43106  16.035878    143.0
12994   0.000000     32.0
10313   0.000000     18.0
26702   0.000000      8.0


In [1152]:
userFeatures['transactionNum'].fillna(0, inplace=True)
userFeatures['transactionNum']=userFeatures['transactionNum'].astype(int)


userFeatures['salaryNum'].fillna(0, inplace=True)
userFeatures['salaryNum']=userFeatures['salaryNum'].astype(int)

# userFeatures['browseNum'].fillna(0, inplace=True)
userFeatures['browseTypes'].fillna(0, inplace=True)
userFeatures['browseTypes']=userFeatures['browseTypes'].astype(int)

userFeatures['browseSubTypes'].fillna(0, inplace=True)
userFeatures['browseSubTypes']=userFeatures['browseSubTypes'].astype(int)

userFeatures['billBanksNum'].fillna(0, inplace=True)
userFeatures['billBanksNum']=userFeatures['billBanksNum'].astype(int)

userFeatures['billNum'].fillna(0, inplace=True)
userFeatures['billNum']=userFeatures['billNum'].astype(int)

userFeatures['salaryTotal'].fillna(0, inplace=True)
userFeatures['transactionAmount'].fillna(0, inplace=True)
# userFeatures['browseNum']=userFeatures['browseNum'].astype(int)





print userFeatures.head()

      sex  profession  education  marriage  citizen_type  transactionNum  \
id                                                                         
3150    1           2          4         1             4               0   
6965    1           2          4         3             2             364   
1265    1           3          4         3             1             419   
6360    1           2          4         3             2               0   
2583    2           2          2         1             1             831   

      transactionAmount  transactionExpense  transactionExpenseMax  \
id                                                                   
3150           0.000000                 NaN                    NaN   
6965        4207.382203         3234.531975              15.366101   
1265        5370.663258         3662.457063              18.256472   
6360           0.000000                 NaN                    NaN   
2583        9800.785996         7064.310678    

In [1153]:
userFeatures.sort_index(inplace=True)
trainOverdue.sort_index(inplace=True)

In [1154]:
def convert2Dummies(df, cln):
    dummies=pd.get_dummies(df[cln]).astype(int)
    dummies.columns=[cln+`c` for c in dummies.columns]

    return pd.concat([df, dummies], axis=1)

In [1155]:
userFeatures=convert2Dummies(userFeatures, 'sex')
userFeatures=convert2Dummies(userFeatures, 'profession')
userFeatures=convert2Dummies(userFeatures, 'education')
userFeatures=convert2Dummies(userFeatures, 'marriage')
userFeatures=convert2Dummies(userFeatures, 'citizen_type')


In [1156]:
# overdueAmount=userFeatures['transactionAmount'].values[trainOverdue.values.ravel()==1]
# plt.hist(overdueAmount, bins=20)
# plt.figure()
# plt.hist(userFeatures['transactionAmount'].values, bins=20)

In [1157]:
print userFeatures.head()

    sex  profession  education  marriage  citizen_type  transactionNum  \
id                                                                       
1     1           2          3         1             3              86   
2     1           2          3         2             1               0   
3     1           4          4         1             4             679   
4     1           4          4         3             2             291   
5     1           2          2         3             1               0   

    transactionAmount  transactionExpense  transactionExpenseMax  \
id                                                                 
1         1156.718031          676.025269              17.157860   
2            0.000000                 NaN                    NaN   
3         7264.831053         4985.957607              15.142957   
4         3293.768106         2129.425722              15.142957   
5            0.000000                 NaN                    NaN   

    

In [1158]:
userFeatures_select=userFeatures.copy()

In [1159]:
X=userFeatures_select.values
imp=Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
X=imp.fit_transform(X)
y=trainOverdue.values.ravel()

In [1160]:
corrmatrix=np.corrcoef(X.T, y)
print corrmatrix[-1,:-1]

[-0.09984565  0.00826746  0.03870428  0.00396767  0.01066433  0.02240804
  0.01833539  0.01928639 -0.052578    0.02922111  0.01526059  0.01430856
  0.00833712 -0.00175276 -0.00548025 -0.01266307 -0.01799713 -0.02566857
  0.01834179  0.01834026 -0.07649229 -0.05819992 -0.03415144  0.01075614
 -0.02471834  0.13775726 -0.01059447 -0.05074661  0.01815592  0.00654069
 -0.01376585 -0.00278987  0.0146259   0.01804614 -0.00943854 -0.0422042
 -0.00684591  0.03592918  0.01793669 -0.00793756  0.00428987 -0.00054338
  0.00666468  0.01036003  0.01804614 -0.02056155  0.01613081 -0.01261837
  0.01202309]


In [1161]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=8)
weights=y_train+1

In [1162]:
scaler=StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[-0.33689176,  2.14097994,  0.99953475, ..., -0.62446927,
        -0.41475882, -0.58884405],
       [-0.33689176,  0.81791615, -0.30181203, ...,  1.60135982,
        -0.41475882, -0.58884405],
       [-0.33689176, -0.50514763, -0.30181203, ...,  1.60135982,
        -0.41475882, -0.58884405],
       ..., 
       [-0.33689176, -0.50514763, -0.30181203, ..., -0.62446927,
        -0.41475882,  1.6982425 ],
       [-0.33689176, -0.50514763, -0.30181203, ..., -0.62446927,
        -0.41475882, -0.58884405],
       [-0.33689176, -0.50514763,  0.99953475, ...,  1.60135982,
        -0.41475882, -0.58884405]])

In [1163]:
factors=abs(np.array(corrmatrix[-1, :-1]))**1.7
for i in xrange(2):
    factors=normalize(factors.reshape(1,-1))
    factors*=abs(corrmatrix[-1, :-1])
print factors

[[  3.75723479e-02   3.72930691e-06   1.12734968e-03   2.46564457e-07
    9.56542360e-06   1.49226075e-04   7.10434140e-05   8.56601232e-05
    3.50207478e-03   3.98500192e-04   3.60217416e-05   2.83827789e-05
    3.84688647e-06   1.19982976e-08   8.14541424e-07   1.80611164e-05
    6.63135899e-05   2.46681477e-04   7.11352217e-05   7.11133437e-05
    1.40196096e-02   5.09992080e-03   7.09518433e-04   9.87367824e-06
    2.14546824e-04   1.23615832e-01   9.33564044e-06   3.07154349e-03
    6.85043852e-05   1.56731612e-06   2.45992353e-05   6.69904677e-08
    3.07825436e-05   6.69842608e-05   6.08832247e-06   1.55298283e-03
    1.85543396e-06   8.56062900e-04   6.54933405e-05   3.20769042e-06
    3.29149911e-07   1.57480312e-10   1.68009678e-06   8.59381603e-06
    6.69842608e-05   1.08556876e-04   4.42260947e-05   1.78263674e-05
    1.49076687e-05]]


In [1164]:
X_train*=factors
X_test*=factors
    

In [1165]:
knnc = KNeighborsClassifier(n_neighbors=10, weights='distance')
knnc.fit(X_train, y_train)
pred=knnc.predict_proba(X_test)[:,1]

In [1166]:
print len(pred[pred>0.5])
print 1.0*sum(y_test)/len(y_test)
print sum(y_test)
print sum(y_train)
print 1.0*len(pred[(pred>0.5)&(y_test==1)])/sum(y_test==1)

p=pred[y_test==1]
n=pred[y_test==0]
print ks_2samp(p,n)

256
0.130575539568
1452
5731
0.0633608815427
Ks_2sampResult(statistic=0.14814798984694044, pvalue=1.1694036526653201e-24)
